# Solving Sudoku Puzzles
In this notebook i will be writing an agent that can solve sudoku puzzles. Im given a 9x9 grid with some fixed values. 
The objective is to fill the empty cells of the grid so that the numbers 1 to 9 appear exactly once in each row,column and 3x3 block of grid

Below is a sample puzzle along with its solution. 

<img src="images/sudoku.png" style="width: 50%;"/>


In [1]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


## Solution
In this next cell i write the sudoku solver using a back tracking algorithm
 

In [2]:
import numpy as np


##################### Setting up all the values needed to set up grid ####################
def crossProduct(A, B):
    "Cross product of elements in A and elements in B."
    return [a+b for a in A for b in B]
#List comprehension 
#Strings can be used the same as arrays 
column   = '123456789'
row     = 'ABCDEFGHI'
possibleValues  = column

gridPosition  = crossProduct(row, column)

unitlist = ([crossProduct(row, c) for c in column] +
            [crossProduct(r, column) for r in row] +
            [crossProduct(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
#Basically creates every column,row and 3*3 grid that a gridposition may belong too


#For every grid posision s as the key, the value of the key value pair is the potenital 
#unit that grid posisitoin s may belong too. 
#for exapmle A1 belongs to column and row with A1, and the top left most 3*3 grid
units = dict((s, [u for u in unitlist if s in u])
             for s in gridPosition)

#Same as above but remove the self occurence of the grid position 
#So remove the current Grid position to get a set of only positions that s belongs too 
peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in gridPosition)


#################### Setting up grid, with values given as inpuit  ##################

def parse_grid(grid):
    
    #To start with every position in the grid can have any value 1 to 9
    #thus the dictionary frist starts with each posisiotn as a key 
    # and the string 123456789 as the value 
    values = dict((s, possibleValues) for s in gridPosition)
    
    #For each key value pair in the grid
    for position,value in grid_values(grid).items():
        #if the value is a possible value and is not valud 
        if value in possibleValues and not valid(values, position, value):
            return False #End the loop and return false, This measn the input
                         #grid values do not meet the constraints required to build it 
                         #Thus the grid is inavlid and not solvable 
                
    #If all the values are assignable and valid, return the dictionary with updated values 
    return values


#Creating the grid with the key as its position and current value as its value
def grid_values(grid):
    #for each value in the grid, only put the value in the arry if it is subset of
    #possible values or 0 (when empty )
    values = [c for c in grid if c in possibleValues or c in '0']
    
    #Return a dictionary, with each gridposition as a key, and its corresponding value 
    #from the list values above. A1 is paired with values[0] A2 with values[1] etc...
    return dict(zip(gridPosition, values))


########################### Adding constraints #########################

#Checking if the value 
def valid(values, position, value):
    #create a tmp string that holds the rest of the values for the current position
    #replace the given value with an empty char so that you are lift with a string 
    #of value 1 to 9 without the given value 
    other_values = values[position].replace(value, '')
    
    #if all the tmpvalues meet the constraints and there is no conflict 
    #for any of the possible values left then the possible value is valid
    #return the dictionary bacl
    if all(constraint(values, position, tmpValue) for tmpValue in other_values):
        return values
    else:
        #if any of the remaning values has a conflict then return false 
        return False

#Constraint is a recursive function that checks if every value is assignable or not,
#if it any point there is a restriction then it will return false, this means the grid is 
#unsolvable 
def constraint(values, position, value):
    
    #if the given value has already been removed from the possible values then nothing 
    #else needs to be done 
    if value not in values[position]:
        return values #return dictionary values

    #remove the given value from the potential value list 
    values[position] = values[position].replace(value,'')
    
    #if by removing the value there is no possible value left
    if len(values[position]) == 0:
        return False #there is a contradiction, each grid position must have a value
                     #if it doesnt then, the grid is unsolvable 
    
    #if the possible list is now only 1 value, then call constraint on this value to check
    #if it is a valud position or not 
    elif len(values[position]) == 1:
        tmpValue = values[position]
        if not all(constraint(values, tmpPosition, tmpValue) for tmpPosition in peers[position]):
            # if there is any contradiction then return false
            return False
        
    #if a unit only has one possible value left, then this must be the value for that unit
    for u in units[position]:
        dplaces = [position for position in u if value in values[position]]
        if len(dplaces) == 0:
            return False #if the length of possibel values is 0 then grid is unsolvable 
        
        elif len(dplaces) == 1:#if length is 1 then this is the position for that value
            if not valid(values, dplaces[0], value): #check if this is a valid position 
                return False
    return values


################################ Solving sudoku ###############################

def solve(grid):
    #first need to create the grid, so pass the values to parse_grid so it can set up 
    #all the grid positions and their possible values 
    tmpGrid = parse_grid(grid)
    return search(tmpGrid)


####################### Search algorithm (Back tracking)######################
def search(values):
    if values is False:
        return False #No solution for this path
    
    if all(len(values[position]) == 1 for position in gridPosition):
        return values #if all grid positions only have one value, then the solution is found
    
    #for each position in the grid, get the position with the smallest length of possible
    #values first, since you are more likely to guess correctly, the less possible values
    #there are. Store the key value pair as value and position 
    value,position = min((len(values[position]), position) 
                     for position in gridPosition if len(values[position]) > 1)
    
    #recursively call search for each value in values, then going onto to a different
    #position and checking that branch recurssively etc...
    #create a copy so that the parent of the current branch is not effected 
    return branch(search(valid(values.copy(), position, value)) for value in values[position])

#branch is called to make sure the path chose is valud,
#for each p in path it checks if p is true, if it is then return p 
#else return false, this path is not valid
def branch(path):
    for p in path:
        if p: 
            return p
        
    return False


####################### Converting ####################

#since we are using a diciontary instead of a list, and possible values are stored as a string
#instead of a list of values, i need to first convert the given grid into a list of string 
def convertToString(matrix):
    mList = matrix.tolist()#convert the grid from a np array into a python 2d list
    tmpStr = "" #tmp string thats empty for now 
    
    #for each value in the list append it to the empty string to create a string of all values
    #that correspond to grid positions
    for y in range(9):
        for x in range(9):
            tmpStr = tmpStr + str(mList[y][x])
        
    return tmpStr

#as the algorithm solves the puzzle using a dictionary and not a 2d list, I must convert the 
#dicitonary into a 2d array
def convertToMatrix(values):
    count = 0 #create a count so that i know which row i am on currently 
    tmp = [] #tmp list to store the convert dictionary in 
    
    t = [] #used to temporaryu hold the row 
    l = [] #used to store all dictionary values in a 1d array
    
    #try creating a 2d array, it it is not possible then the sudoku is inavlid
    #if valid return the 2d array, elsereturn -1 
    try:
        if all(len(values[s]) == 1 for s in gridPosition):
            #Append all key values inot l so they can be refered to in the next loop 
            for i in values:
                l.append(i)
            
            #for each key in l append the corresponding value from the dictionary, for the 
            #current row 
            for x in range(9):
                for y in range(9):
                    t.append(int((values[l[count+y]])))
                tmp.append(t) #append this row to tmp
                t = [] #clear t so that we can use it for the next row 
                count+=9
            return tmp
    except:
        return -1
        
        
def sudoku_solver(puzzle):
    Sgrid = convertToString(puzzle) # convert 2d arry into a string 
    
    #if convertingToMatrix returns -1, it is inavlid thus return a 2d array of -1
    if convertToMatrix(solve(Sgrid)) == -1:
        return np.matrix([[-1]*9]*9)
    #else return the 2d array as a numpy matrix 
    else:
        return np.matrix(convertToMatrix(solve(Sgrid)))



## Testing the code
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers

In the next cell you will find a time stamp to show how long it took my algorithm to find a solution as well as the solution itself. An output of a matrix full of -1 means that the algorithm could not find a solution to the problem.

In [4]:
SKIP_TESTS = False

if not SKIP_TESTS:
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break

Testing very_easy sudokus
This is very_easy sudoku number 0
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]]
This is your solution for very_easy sudoku number 0
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.008987000000000078 seconds to solve.

This is very_easy sudoku number 1
[[0 9 3 1 5 2 6 0 8]
 [8 6 2 7 0 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 0 3 6]
 [5 0 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]]
This is your solution for very_easy sudoku number 1
[[4 9 3 1 5 2 6 7 8]
 [8 6 2 7 4 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 5 3 6]
 [5 2 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]


This is your solution for easy sudoku number 1
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.004342000000000068 seconds to solve.

This is easy sudoku number 2
[[8 5 7 4 3 1 9 2 6]
 [2 6 4 0 5 9 3 1 7]
 [1 3 9 2 7 6 0 8 5]
 [4 8 6 9 2 3 7 5 1]
 [7 2 1 8 4 5 6 9 3]
 [5 9 0 6 1 7 8 4 2]
 [3 4 8 1 6 2 5 7 9]
 [0 1 5 0 9 4 2 3 8]
 [9 7 2 3 1 8 1 6 4]]
This is your solution for easy sudoku number 2
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]
Is your solution correct?
Yes! Correct 

This is your solution for hard sudoku number 6
[[1 8 9 4 3 2 5 6 7]
 [4 3 7 9 5 6 8 1 2]
 [2 5 6 7 1 8 4 9 3]
 [7 4 1 8 2 9 6 3 5]
 [3 9 5 1 6 4 2 7 8]
 [6 2 8 5 7 3 1 4 9]
 [5 6 2 3 9 1 7 8 4]
 [9 7 4 6 8 5 3 2 1]
 [8 1 3 2 4 7 9 5 6]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.00903799999999988 seconds to solve.

This is hard sudoku number 7
[[0 9 0 0 3 1 8 0 0]
 [1 0 5 0 0 0 0 0 0]
 [8 0 0 9 0 0 0 0 7]
 [0 0 0 7 0 0 0 3 0]
 [0 0 9 0 2 0 7 0 0]
 [0 2 0 0 0 0 0 0 5]
 [0 0 0 0 0 0 0 0 0]
 [0 4 0 2 0 8 0 6 0]
 [0 0 8 4 0 5 3 7 0]]
This is your solution for hard sudoku number 7
[[2 9 7 5 3 1 8 4 6]
 [1 6 5 8 4 7 9 2 3]
 [8 3 4 9 6 2 5 1 7]
 [4 8 6 7 5 9 2 3 1]
 [3 5 9 1 2 6 7 8 4]
 [7 2 1 3 8 4 6 9 5]
 [9 7 2 6 1 3 4 5 8]
 [5 4 3 2 7 8 1 6 9]
 [6 1 8 4 9 5 3 7 2]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.00799799999999995 seconds to solve.

This is hard sudoku number 8
[[0 0 2 0 0 0 0 0 4]
 [0 5 0 0 1 3 7 0 0]
 [7 9 0 0 0 0 0 5 0]
 [0 0

In [ ]:
|